### AISV_801 Natural Language Processing ###
#### Instructor Joseph Meyer #### 
Lab7: Fine tune an LLM on a custom dataset
        Helper script
        Let’s start witht lamini
        https://huggingface.co/MBZUAI/LaMini-Cerebras-590M
        Use the guanaco dataset
        https://huggingface.co/datasets/timdettmers/openassistant-guanaco

Bobby Wen

In [1]:
!pip install -U xformers

In [2]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="MBZUAI/LaMini-Cerebras-590M")

In [3]:
import torch
import transformers
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, Trainer, TrainingArguments
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Cerebras-590M")
# model = AutoModelForCausalLM.from_pretrained("MBZUAI/LaMini-Cerebras-590M")

tokenizer = AutoTokenizer.from_pretrained("MBZUAI/LaMini-Cerebras-111M")
model = AutoModelForCausalLM.from_pretrained("MBZUAI/LaMini-Cerebras-111M")

In [4]:
!pip install transformers

In [5]:
# def query(payload):
#     inputs = tokenizer(payload, return_tensors="pt")

#     ###model_outputs = model.generate(**inputs, max_new_tokens=5, return_dict_in_generate=True, output_scores=True)
#     model_outputs = model.generate(**inputs,return_dict_in_generate=True, output_scores=True)

#     generated_tokens_ids = model_outputs.sequences[0]

#     response = tokenizer.decode(generated_tokens_ids)
#     return response

In [6]:
from datasets import load_dataset

dataset = load_dataset("timdettmers/openassistant-guanaco")

Found cached dataset json (C:/Users/b1/.cache/huggingface/datasets/timdettmers___json/timdettmers--openassistant-guanaco-6126c710748182cf/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [8]:
dataset["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='json', config_name='timdettmers--openassistant-guanaco', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=15808615, num_examples=9846, shard_lengths=None, dataset_name='json'), 'test': SplitInfo(name='test', num_bytes=839966, num_examples=518, shard_lengths=None, dataset_name='json')}, download_checksums={'https://huggingface.co/datasets/timdettmers/openassistant-guanaco/resolve/831dabac2283d99420cda0b673d7a2a43849f17a/openassistant_best_replies_train.jsonl': {'num_bytes': 20877686, 'checksum': None}, 'https://huggingface.co/datasets/timdettmers/openassistant-guanaco/resolve/831dabac2283d99420cda0b673d7a2a43849f17a/openassistant_best_replies_eval.jsonl': {'num_bytes': 1105272, 'checksum': None}}, download_size=21982958, post_processing_size=None, dataset_size=16648581, size_in_

In [9]:
dataset['train'].features

{'text': Value(dtype='string', id=None)}

In [10]:
dataset['train'].shape

(9846, 1)

In [11]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-9): 10 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): GELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [12]:
X_train = dataset["train"]
X_test = dataset["test"]

In [13]:
def tokenize(examples):
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_train_dataset = X_train.map(tokenize, batched=True)
tokenized_test_dataset = X_test.map(tokenize, batched=True)


Loading cached processed dataset at C:\Users\b1\.cache\huggingface\datasets\timdettmers___json\timdettmers--openassistant-guanaco-6126c710748182cf\0.0.0\8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96\cache-90858038a537434d.arrow
Loading cached processed dataset at C:\Users\b1\.cache\huggingface\datasets\timdettmers___json\timdettmers--openassistant-guanaco-6126c710748182cf\0.0.0\8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96\cache-ada3480ec28864ca.arrow


In [14]:
!pip install transformers[torch]

In [15]:
!pip install accelerate -U

In [16]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [17]:
data_collator = DataCollatorWithPadding(tokenizer)

In [18]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    pred = pred.predictions.argmax(-1)
    f1 = f1_score(labels, pred, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [19]:
training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="./LLM_trainer",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  logging_dir="./train_logs",
                                  logging_steps=500,
                                 # save_steps=2000,
                                  #evaluation_strategy="steps",
                                #  evaluation_strategy="epoch",
                                  eval_steps=500,
                                  save_total_limit=2,
                                 )

trainer = Trainer(model=model,
                  tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  # train_dataset=tokenized_dataset["train"],
                  # eval_dataset=tokenized_dataset["test"],
                  train_dataset=tokenized_train_dataset,
                  eval_dataset=tokenized_test_dataset,
                  #compute_metrics=compute_metrics
                  #compute_metrics=compute_f1
                 )

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 10
logging_steps = len(transcripts_encoded["train"]) // batch_size
logging_steps = 1000 // batch_size
model_name = f"{model_checkpoint}-finetuned-transcripts"

training_args = TrainingArguments(output_dir=model_name,
                                 num_train_epochs=2,
                                 learning_rate=2e-5,
                                 per_device_train_batch_size=batch_size,
                                 per_device_eval_batch_size=batch_size,
                                 weight_decay=0.01,
                                 evaluation_strategy="epoch",
                                 disable_tqdm=False,
                                 logging_steps=logging_steps,
                                 push_to_hub=False,
                                 log_level="error")

from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                 compute_metrics=compute_metrics,
                  train_dataset=tokenized_train_dataset,
                  eval_dataset=tokenized_test_dataset,                 tokenizer=tokenizer)


In [20]:
trainer.train()

C:\Users\b1\.conda\envs\Tensorflow_env\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
trainer.evaluate()